In [1]:
import numpy as np
import matplotlib.pyplot as plt
import moviepy.editor as mpy
import skimage.transform
from IPython.display import Image, display

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras.losses as kls

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [2]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!git clone https://github.com/ntasfi/PyGame-Learning-Environment "/content/drive/MyDrive/Colab Notebooks/dl/lecture15/rep"

Cloning into '/content/drive/MyDrive/Colab Notebooks/dl/lecture15/rep'...
remote: Enumerating objects: 1118, done.
remote: Total 1118 (delta 0), reused 0 (delta 0), pack-reused 1118 (from 1)
Receiving objects: 100% (1118/1118), 8.06 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [16]:
import os

# 切换工作目录到在 Drive 中克隆的仓库路径
repo_path = "/content/drive/MyDrive/Colab Notebooks/dl/lecture15/rep"
if os.path.exists(repo_path):
    os.chdir(repo_path)
    print(f"Changed directory to: {os.getcwd()}")
else:
    print(f"Path not found: {repo_path}. Please check if the drive is mounted and the path is correct.")

Changed directory to: /content/drive/MyDrive/Colab Notebooks/dl/lecture15/rep


In [8]:
# 在当前目录下安装 PLE 环境
!pip install -e .

Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/dl/lecture14/rep
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ple


In [9]:
# Install pygame
!pip install pygame

In [34]:
#切換回路徑
os.chdir("/content/drive/MyDrive/Colab Notebooks/dl/lecture15")

In [35]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"  # this line make pop-out window not appear
from ple.games.flappybird import FlappyBird
from ple import PLE

game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game
env.reset_game()

test_game = FlappyBird()
test_env = PLE(test_game, fps=30, display_screen=False)
test_env.reset_game()

In [38]:
path = './movie_f'
if not os.path.exists(path):
    os.makedirs(path)

In [39]:
hparas = {
    'image_size': 84,
    'num_stack': 4,
    'action_dim': len(env.getActionSet()),
    'hidden_size': 256,
    'lr': 0.0001,
    'gamma': 0.99,
    'lambda': 0.95,
    'clip_val': 0.2,
    'ppo_epochs': 8,
    'test_epochs': 1,
    'num_steps': 512,
    'mini_batch_size': 64,
    'target_reward': 200,
    'max_episode': 30000,
}

In [40]:
# Please do not modify this method
def make_anim(images, fps=60, true_image=False):
    duration = len(images) / fps

    def make_frame(t):
        try:
            x = images[int(len(images) / duration * t)]
        except:
            x = images[-1]

        if true_image:
            return x.astype(np.uint8)
        else:
            return ((x + 1) / 2 * 255).astype(np.uint8)

    clip = mpy.VideoClip(make_frame, duration=duration)
    clip.fps = fps

    return clip

In [41]:
def preprocess_screen(screen):
    screen = skimage.transform.rotate(screen, -90, resize=True)
    screen = screen[:400, :]
    screen = skimage.transform.resize(screen, [hparas['image_size'], hparas['image_size'], 1])
    return screen.astype(np.float32)

def frames_to_state(input_frames):
    if(len(input_frames) == 1):
        state = np.concatenate(input_frames*4, axis=-1)
    elif(len(input_frames) == 2):
        state = np.concatenate(input_frames[0:1]*2 + input_frames[1:]*2, axis=-1)
    elif(len(input_frames) == 3):
        state = np.concatenate(input_frames + input_frames[2:], axis=-1)
    else:
        state = np.concatenate(input_frames[-4:], axis=-1)

    return state

In [42]:
class ActorCriticNetwork(tf.keras.Model):
    def __init__(self, hparas):
        super().__init__()

        self.feature_extractor = tf.keras.Sequential([
          # Convolutional Layers
          tf.keras.layers.Conv2D(filters=32, kernel_size=8, strides=4),
          tf.keras.layers.ReLU(),
          tf.keras.layers.Conv2D(filters=64, kernel_size=4, strides=2),
          tf.keras.layers.ReLU(),
          tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1),
          tf.keras.layers.ReLU(),
          # Embedding Layers
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(hparas['hidden_size']),
          tf.keras.layers.ReLU(),
        ])

        # Actor Network
        self.actor = tf.keras.layers.Dense(hparas['action_dim'], activation='softmax')
        # Critic Network
        self.critic = tf.keras.layers.Dense(1, activation = None)

    def call(self, input):
        x = self.feature_extractor(input)
        action_logits = self.actor(x)
        value = self.critic(x)
        return action_logits, value

In [43]:
class Agent():
    def __init__(self, hparas):
        self.gamma = hparas['gamma']
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=hparas['lr'])
        self.actor_critic = ActorCriticNetwork(hparas)
        self.clip_pram = hparas['clip_val']

    def ppo_iter(self, mini_batch_size, states, actions, log_probs, returns, advantage):
        batch_size = states.shape[0]
        for _ in range(batch_size // mini_batch_size):
            rand_ids = tf.convert_to_tensor(np.random.randint(0, batch_size, mini_batch_size), dtype=tf.int32)
            yield tf.gather(states, rand_ids), tf.gather(actions, rand_ids), tf.gather(log_probs, rand_ids), \
             tf.gather(returns, rand_ids), tf.gather(advantage, rand_ids)

    def ppo_update(self, ppo_epochs, mini_batch_size, states, actions, log_probs, discount_rewards, advantages):
        total_actor_loss = 0
        total_critic_loss = 0
        for _ in range(ppo_epochs):
            for state, action, old_log_probs, reward, advantage in self.ppo_iter(mini_batch_size, states, actions, log_probs, discount_rewards, advantages):
                reward = tf.expand_dims(reward, axis=-1)

                with tf.GradientTape() as tape:
                    prob, value = self.actor_critic(state, training=True)
                    dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
                    entropy = tf.math.reduce_mean(dist.entropy())
                    new_log_probs = dist.log_prob(action)

                    # PPO ratio
                    ratio = tf.math.exp(new_log_probs - old_log_probs)
                    surr1 = ratio * advantage
                    surr2 = tf.clip_by_value(ratio, 1.0 - self.clip_pram, 1.0 + self.clip_pram) * advantage

                    actor_loss = tf.math.negative(tf.math.reduce_mean(tf.math.minimum(surr1, surr2))) - 0.1 * entropy
                    critic_loss = 0.5 * tf.math.reduce_mean(tf.keras.losses.mse(reward, value))

                    total_loss = actor_loss + critic_loss

                # single optimizer
                grads = tape.gradient(total_loss, self.actor_critic.trainable_variables)
                self.optimizer.apply_gradients(zip(grads, self.actor_critic.trainable_variables))

                total_actor_loss += actor_loss
                total_critic_loss += critic_loss
        return total_actor_loss, total_critic_loss

In [44]:
# https://arxiv.org/pdf/1506.02438.pdf
# Equation 16
def compute_gae(rewards, masks, values, gamma, LAMBDA):
    gae = 0
    returns = []
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] * masks[i] - values[i]
        gae = delta + gamma * LAMBDA * masks[i] * gae
        returns.append(gae + values[i])

    returns.reverse()
    return returns

Testing Environment

In [45]:
def test_reward(test_env, agent):
    total_reward = 0
    # Reset the environment
    test_env.reset_game()
    input_frames = [preprocess_screen(test_env.getScreenGrayscale())]

    while not test_env.game_over():

        state = frames_to_state(input_frames)
        state = tf.expand_dims(state, axis=0)
        prob, value = agent.actor_critic(state)

        action = np.argmax(prob[0].numpy())
        reward = test_env.act(test_env.getActionSet()[action])
        total_reward += reward

        input_frames.append(preprocess_screen(test_env.getScreenGrayscale()))

    return total_reward

Training

In [52]:
agent = Agent(hparas)
max_episode = hparas['max_episode']
test_per_n_episode = 10
force_save_per_n_episode = 1000
early_stop_reward = 10

start_s = 0
best_reward = -5.0

# Create directories for saving models and checkpoints if they don't exist
if not os.path.exists('./save/Actor'):
    os.makedirs('./save/Actor')
if not os.path.exists('./save/checkpoints'):
    os.makedirs('./save/checkpoints')

checkpoint = tf.train.Checkpoint(
    actor_critic = agent.actor_critic,
    optimizer = agent.optimizer,
)

# Load from old checkpoint
checkpoint.restore('/content/drive/MyDrive/Colab Notebooks/dl/lecture15/save/checkpoints/ckpt-2')

In [53]:
ep_reward = []
total_avgr = []
early_stop = False
avg_rewards_list = []

env.reset_game()

for s in range(0, max_episode):
    if early_stop == True:
        break

    rewards = []
    states = []
    actions = []
    log_probs = []
    masks = []
    values = []

    display_frames = [env.getScreenRGB()]
    input_frames = [preprocess_screen(env.getScreenGrayscale())]

    for step in range(hparas['num_steps']):

        state = frames_to_state(input_frames)
        state = tf.expand_dims(state, axis=0)
        prob, value = agent.actor_critic(state)

        dist = tfp.distributions.Categorical(probs=prob[0], dtype=tf.float32)
        action = dist.sample(1)
        log_prob = dist.log_prob(action)

        reward = env.act(env.getActionSet()[int(action.numpy())])

        done = env.game_over()

        states.append(state)
        actions.append(action)
        values.append(value[0])
        log_probs.append(log_prob)
        rewards.append(tf.convert_to_tensor(reward, dtype=tf.float32))
        masks.append(tf.convert_to_tensor(1-int(done), dtype=tf.float32))

        display_frames.append(env.getScreenRGB())
        input_frames.append(preprocess_screen(env.getScreenGrayscale()))

        if done:
            env.reset_game()
            input_frames = [preprocess_screen(env.getScreenGrayscale())]

    _, next_value = agent.actor_critic(state)
    values.append(next_value[0])

    returns = compute_gae(rewards, masks, values, hparas['gamma'], hparas['lambda'])

    returns = tf.concat(returns, axis=0)
    log_probs = tf.concat(log_probs, axis=0)
    values = tf.concat(values, axis=0)
    states = tf.concat(states, axis=0)
    actions = tf.concat(actions, axis=0)
    advantage = returns - values[:-1]

    a_loss, c_loss = agent.ppo_update(hparas['ppo_epochs'], hparas['mini_batch_size'], states, actions, log_probs, returns, advantage)
    print('[Episode %d]  Actor loss: %.5f, Critic loss: %.5f' % (s, a_loss, c_loss))

    if s % test_per_n_episode == 0:
        # test agent hparas['test_epochs'] times to get the average reward
        avg_reward = np.mean([test_reward(test_env, agent) for _ in range(hparas['test_epochs'])])
        print("Test average reward is %.1f, Current best average reward is %.1f\n" % (avg_reward, best_reward))
        avg_rewards_list.append(avg_reward)

        if avg_reward > best_reward:
            best_reward = avg_reward
            agent.actor_critic.save('./save/Actor/model_actor_{}_{}.keras'.format(s, avg_reward))
            checkpoint.save(file_prefix = './save/checkpoints/ckpt')

    if s % force_save_per_n_episode == 0:
        agent.actor_critic.save('./save/Actor/model_actor_{}_{}.keras'.format(s, avg_reward))
        checkpoint.save(file_prefix = './save/checkpoints/ckpt')
        clip = make_anim(display_frames, fps=60, true_image=True).rotate(-90)
        clip.write_videofile("movie_f/{}_demo-{}.webm".format('Lab15', s), fps=60)
        display(clip.ipython_display(fps=60, autoplay=1, loop=1, maxduration=120))

    if best_reward >= early_stop_reward:
        early_stop = True

  reward = env.act(env.getActionSet()[int(action.numpy())])



[Episode 0]  Actor loss: 83.23114, Critic loss: 57.00113
Test average reward is -5.0, Current best average reward is -5.0

Moviepy - Building video movie_f/Lab15_demo-0.webm.
Moviepy - Writing video movie_f/Lab15_demo-0.webm



Moviepy - Done !
Moviepy - video ready movie_f/Lab15_demo-0.webm
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


[Episode 1]  Actor loss: 59.53554, Critic loss: 34.03883
[Episode 2]  Actor loss: 16.90125, Critic loss: 11.32213
[Episode 3]  Actor loss: 5.14103, Critic loss: 6.25601
[Episode 4]  Actor loss: 4.08655, Critic loss: 5.87614
[Episode 5]  Actor loss: 0.48981, Critic loss: 4.27373
[Episode 6]  Actor loss: 4.72045, Critic loss: 3.53775
[Episode 7]  Actor loss: 2.35840, Critic loss: 3.90467
[Episode 8]  Actor loss: -6.32339, Critic loss: 4.60197
[Episode 9]  Actor loss: -6.38582, Critic loss: 4.57783
[Episode 10]  Actor loss: 0.27117, Critic loss: 3.49852
Test average reward is -5.0, Current best average reward is -5.0

[Episode 11]  Actor loss: 0.52871, Critic loss: 2.99603
[Episode 12]  Actor loss: -3.54149, Critic loss: 2.60895
[Episode 13]  Actor loss: -4.14063, Critic loss: 2.59569
[Episode 14]  Actor loss: -3.54859, Critic loss: 2.44528
[Episode 15]  Actor loss: 4.26943, Critic loss: 3.38265
[Episode 16]  Actor loss: -10.38640, Critic loss: 3.37755
[Episode 17]  Actor loss: 0.96797, C

...

[Episode 2001]  Actor loss: -6.27127, Critic loss: 5.38208
[Episode 2002]  Actor loss: -4.93918, Critic loss: 4.95503
[Episode 2003]  Actor loss: -20.17994, Critic loss: 3.17081
[Episode 2004]  Actor loss: -15.90139, Critic loss: 6.45377
[Episode 2005]  Actor loss: -4.49443, Critic loss: 3.72186
[Episode 2006]  Actor loss: 12.53650, Critic loss: 9.73710
[Episode 2007]  Actor loss: 17.24653, Critic loss: 4.40480
[Episode 2008]  Actor loss: 31.52471, Critic loss: 5.63278
[Episode 2009]  Actor loss: 2.31183, Critic loss: 3.16335
[Episode 2010]  Actor loss: 2.55777, Critic loss: 1.65306
Test average reward is -2.0, Current best average reward is 7.0

[Episode 2011]  Actor loss: -1.04828, Critic loss: 0.80827
[Episode 2012]  Actor loss: -1.71592, Critic loss: 1.63213
[Episode 2013]  Actor loss: -24.92985, Critic loss: 1.87236
[Episode 2014]  Actor loss: -9.50175, Critic loss: 2.74328
[Episode 2015]  Actor loss: -20.95340, Critic loss: 2.77202
[Episode 2016]  Actor loss: -14.13503, Critic lo

Moviepy - Done !
Moviepy - video ready movie_f/Lab15_demo-3000.webm
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


[Episode 3001]  Actor loss: -11.86368, Critic loss: 3.86108
[Episode 3002]  Actor loss: -2.52984, Critic loss: 6.39609
[Episode 3003]  Actor loss: -31.31890, Critic loss: 3.76285
[Episode 3004]  Actor loss: -23.02593, Critic loss: 3.92442
[Episode 3005]  Actor loss: -8.11767, Critic loss: 3.81226
[Episode 3006]  Actor loss: -38.35426, Critic loss: 3.78426
[Episode 3007]  Actor loss: -46.39825, Critic loss: 3.28293
[Episode 3008]  Actor loss: -31.28071, Critic loss: 2.65152
[Episode 3009]  Actor loss: -25.58754, Critic loss: 7.70019
[Episode 3010]  Actor loss: -23.82731, Critic loss: 8.05724
Test average reward is -3.0, Current best average reward is 9.0

[Episode 3011]  Actor loss: -24.04722, Critic loss: 8.99214
[Episode 3012]  Actor loss: -16.57180, Critic loss: 3.60672
[Episode 3013]  Actor loss: -17.33155, Critic loss: 3.03680
[Episode 3014]  Actor loss: 4.85846, Critic loss: 6.97609


KeyboardInterrupt: 

#### 訓練總結
這次notebook用深度強化學習訓練flappy bird，總共訓練了進3000個epoch，最高得分達到了9分，但是從生成的遊戲視頻上來看agent的表現似乎不太好，沒有普通強化學習進步得明顯。
此外我發現Actor loss 和 Critic loss 都呈現出一個先下降然後再上升波動的趨勢，雖然訓練後期兩個loss都出現了明顯上升，但是agent的得分一直有進步，這可能是因為遊戲環境反饋的變化，但是loss函數仍然在用過去的經驗計算loss，所以數值上loss
不降反升，但是agent卻是在學習到有用的行為。